# Regression with multiple input variables

### Use Vertorization to implement multiple linear regression

#### Multiple features (variables)

| Size in feet^2 ($x_1$) | # of bedrooms ($x_2$)| # of floors ($x_3$) | Age of home ($x_4$) | Price ($y$) |
|----------------|--------------|-------------|-------------|-------|
| 2104           | 5            | 1           | 45          | 460   |
| 1416           | 3            | 2           | 40          | 232   |
| 852            | 2            | 1           | 35          | 178   |
| ...            | ...          | ...         | ...         | ...   |


where:

* $x^{(i)}$ is the input (features) of the $i^{th}$ training example.
    $x^{(1)}$ = [2104, 5, 1, 45]
* $ n $ is the number of features, number of columns in the matrix
    $ n = 4 $
* $ m $ is the number of training examples, number of rows in the matrix
* $x_j$ is the input (features) of the $j^{th}$ feature.
    $x_1$ = [2104, 1416, 852, ...]
* $x^{(i)}_j$ is the input (features) of the $j^{th}$ feature of the $i^{th}$ training example.
    $x^{(1)}_1$ = 2104

The formula for model regresion is: $f_w,_b(x) = w*x + b$

for the multiple features the formula is: 



$$f_w,_b(x) = w_1*x_1 + w_2*x_2 + ... + w_n*x_n + 1b$$
$$ f_{\vec{W},b}(\vec{X}) = \vec{W}\cdot\vec{X} + b $$

#### Vectorization


$$ \vec{W} = \begin{bmatrix} w_1 \\ w_2 \\ \vdots \\ w_n \end{bmatrix} \quad $$
$$ \vec{X} = \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{bmatrix} \quad $$
$$ \vec{W}\cdot\vec{X} = w_1*x_1 + w_2*x_2 + ... + w_n*x_n $$


#### Without vectorization

$$ f_{\vec{W},b}(\vec{X}) = \vec{W}\cdot\vec{X} + b = \sum_{j=1}^{n} w_j*x_j + b $$

#### Gradient Descent

$$ repeat \{ $$
$$ \vec{W} = \vec{W} - \alpha * \dfrac{\partial}{\partial \vec{W}} J(\vec{W},b) $$
$$ b = b - \alpha * \dfrac{\partial}{\partial b} J(\vec{W},b) $$
$$ \} $$
$$ \dfrac{\partial}{\partial \vec{W}} J(\vec{W},b) = \dfrac{1}{m} \sum_{i=1}^{m} (\vec{W}\cdot\vec{X}^{(i)} + b - y^{(i)})\vec{X}^{(i)} $$
$$ \dfrac{\partial}{\partial b} J(\vec{W},b) = \dfrac{1}{m} \sum_{i=1}^{m} (\vec{W}\cdot\vec{X}^{(i)} + b - y^{(i)}) $$

An alternative to gradient descent is the **Normal Equation**:

* No need to choose learning rate, Solve for $w$, $b$ with out iteration
* Only for linear regression
* Doesn't generalize to other models like logistic regression
* Slow if $n$ is very large

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [13]:
# data same of the previous example in the table

X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])
print(X_train)
print(y_train)

[[2104    5    1   45]
 [1416    3    2   40]
 [ 852    2    1   35]]
[460 232 178]


In [14]:
# random initialization of the parameters
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(b_init)
print(w_init)

785.1811367994083
[  0.39133535  18.75376741 -53.36032453 -26.42131618]


In [15]:
def predict_without_vectorization(X, w, b):
    """implementation of the prediction
    Args:
        X: input matrix
        W: weights vector
        b: bias
    returns:
        vector of predictions
    """
    # number of training examples
    m = X.shape[0]
    # number of features
    n = X.shape[1]
    # vector of predictions
    y_hat = np.zeros(m)
    
    for i in range(m):
        for j in range(n):
            y_hat[i] += X[i][j] * w[j]
        y_hat[i] += b
    return y_hat
    

def predict(X, w, b):
    """Vectorized implementation of the prediction
    Args:
        X: input matrix
        W: weights vector
        b: bias
        
    Returns: 
        vector of predictions
    """
    y_hat = np.dot(X, w) + b
    return y_hat

In [16]:
# prediction with the random parameters
assert np.allclose(predict(X_train, w_init, b_init), predict_without_vectorization(X_train, w_init, b_init))   

print(f'prediction: {predict(X_train, w_init, b_init)}')
print(f'prediction without vectorization: {predict_without_vectorization(X_train, w_init, b_init)}')

prediction: [459.99999762 231.99999837 177.99999899]
prediction without vectorization: [459.99999762 231.99999837 177.99999899]


In [17]:
### Cost function with multiple variables
def compute_cost(X, y, w, b):
    """Compute the cost function
    Args:
        X: input matrix
        y: target vector
        w: weights vector
        b: bias
        
    Returns:
        cost function
    """
    m = X.shape[0]
    y_hat = predict(X, w, b)
    cost = (1/(2*m)) * np.sum((y_hat - y)**2)
    return cost

def compute_cost_without_vectorization(X, y, w, b):
    """Compute the cost function without vectorization
    Args:
        X: input matrix
        y: target vector
        w: weights vector
        b: bias
        
    Returns:
        cost function
    """
    m = X.shape[0]
    y_hat = predict_without_vectorization(X, w, b)
    cost = 0
    for i in range(m):
        cost = cost + (y_hat[i] - y[i])**2
    # divide by 2m
    cost = cost / (2*m)
    return cost

In [18]:
# compute gradient multiple variables

def compute_gradient(X, y, w, b, alpha):
    """Compute the gradient of the cost function
    
        args:
            X: input matrix
            y: target vector
            w: weights vector
            b: bias
        returns:
            w: weights vector gradient
            b: bias gradient
            cost: cost function
    """
    m = X.shape[0]
    cost = compute_cost(X, y, w, b)
    dw = (1/m) * (predict(X, w, b) - y) @ X
    db = (1/m) * sum(predict(X, w, b) - y)
    #print('dw: ', dw)
    #print('db: ', db)
    w_temp = w - alpha * dw
    b_temp = b - alpha * db
    return w_temp, b_temp, cost
    
def compute_gradient_without_vectorization(X, y, w, b, alpha):
    """Compute the gradient of the cost function
    
        args:
            X: input matrix
            y: target vector
            w: weights vector
            b: bias
        returns:
            w: weights vector gradient
            b: bias gradient
            cost: cost function
    """
    m = X.shape[0]
    n = X.shape[1]
    dw = np.zeros(n)
    db = 0
    y_hat = predict_without_vectorization(X, w, b)
    # cal partial derivatives
    for i in range(m):
        for j in range(n):
            dw[j] += (y_hat[i] - y[i]) * X[i][j]
        db += (y_hat[i] - y[i])
    
    # update parameters and divide by m
    b_temp = b - (alpha * db) / m
    w_temp = np.zeros(n)
    for j in range(n):
        w_temp[j] = w[j] - (alpha * dw[j]) / m
        
    return w_temp, b_temp, compute_cost_without_vectorization(X, y, w, b)
    

In [19]:
# gradient descent multiple variables
alpha = 0.000001
print(f'alpha: {alpha}')
print('Gradient descent multiple variables, vectorized implementation')
print('-----------------------------------')
print(compute_gradient(X_train, y_train, w_init, b_init, alpha))
print('Gradient descent multiple variables, without vectorization')
print('-----------------------------------')
print(compute_gradient_without_vectorization(X_train, y_train, w_init, b_init, alpha))

alpha: 1e-06
Gradient descent multiple variables, vectorized implementation
-----------------------------------
(array([  0.39133535,  18.75376741, -53.36032453, -26.42131618]), 785.18113679941, 1.5578904045996674e-12)
Gradient descent multiple variables, without vectorization
-----------------------------------
(array([  0.39133535,  18.75376741, -53.36032453, -26.42131618]), 785.18113679941, 1.5578904428966628e-12)


In [20]:
def compute_gradient_iter(X, y, w, b, alpha, iterations, gd_fn):
    """Compute the gradient of the cost function
    
        args:
            X: input matrix
            y: target vector
            w: weights vector
            b: bias
            alpha: learning rate
            iterations: number of iterations
            cost_fn: cost function
        returns:
            w: weights vector gradient
            b: bias gradient
            cost: cost function
    """
    # cal time of execution
    start = time.time()
    costs = []
    for i in range(iterations):
        w, b, cost = gd_fn(X, y, w, b, alpha)
        costs.append(cost)
        if i % 100 == 0:
            print(f'cost at iteration {i}: {cost}, w: {w}, b: {b}')
    end = time.time()
    print(f'execution time: {end - start}')
    return w, b, costs

In [21]:
w_initial = np.zeros_like(w_init)
b_initial = 0
alpha = 5.0e-7
iterations = 1000

w, b, costs = compute_gradient_iter(X_train, y_train, w_initial, b_initial, alpha, iterations, compute_gradient)
print('-----------------------------------')
print(f'w: {w}')
print(f'b: {b}')
y_hat = predict(X_train, w, b)
print(f'prediction: {y_hat}')
print(f'original: {y_train}')

Gradient descent multiple variables, vectorized implementation
cost at iteration 0: 49518.0, w: [2.41334667e-01 5.58666667e-04 1.83666667e-04 6.03500000e-03], b: 0.000145
cost at iteration 100: 696.0010595124638, w: [ 0.20235171  0.00079796 -0.00099658 -0.00219736], b: -0.00011985961877688935
cost at iteration 200: 694.9313476914755, w: [ 0.20253446  0.00112715 -0.00214349 -0.00940619], b: -0.0003596578183953631
cost at iteration 300: 693.8709864577188, w: [ 0.2027164   0.00145611 -0.00328876 -0.01658286], b: -0.0005983240279392168
cost at iteration 400: 692.819893023782, w: [ 0.20289753  0.00178484 -0.00443238 -0.02372751], b: -0.000835863270686938
cost at iteration 500: 691.7779853352556, w: [ 0.20307785  0.00211335 -0.00557437 -0.03084027], b: -0.0010722805476294614
cost at iteration 600: 690.7451820642364, w: [ 0.20325736  0.00244162 -0.00671473 -0.0379213 ], b: -0.001307580837569055
cost at iteration 700: 689.7214026029071, w: [ 0.20343608  0.00276967 -0.00785347 -0.04497072], b: 

In [22]:
w_initial = np.zeros_like(w_init)
b_initial = 0
alpha = 5.0e-7
iterations = 1000

w, b, costs = compute_gradient_iter(X_train, y_train, w_initial, b_initial, alpha, iterations, compute_gradient_without_vectorization)
print('-----------------------------------')
print(f'w: {w}')
print(f'b: {b}')
y_hat = predict(X_train, w, b)
print(f'prediction: {y_hat}')
print(f'original: {y_train}')

Gradient descent multiple variables, vectorized implementation
cost at iteration 0: 49518.0, w: [2.41334667e-01 5.58666667e-04 1.83666667e-04 6.03500000e-03], b: 0.000145
cost at iteration 100: 696.0010595124644, w: [ 0.20235171  0.00079796 -0.00099658 -0.00219736], b: -0.00011985961877688931
cost at iteration 200: 694.9313476914762, w: [ 0.20253446  0.00112715 -0.00214349 -0.00940619], b: -0.00035965781839536286
cost at iteration 300: 693.8709864577195, w: [ 0.2027164   0.00145611 -0.00328876 -0.01658286], b: -0.0005983240279392168
cost at iteration 400: 692.8198930237817, w: [ 0.20289753  0.00178484 -0.00443238 -0.02372751], b: -0.0008358632706869382
cost at iteration 500: 691.7779853352548, w: [ 0.20307785  0.00211335 -0.00557437 -0.03084027], b: -0.0010722805476294612
cost at iteration 600: 690.7451820642369, w: [ 0.20325736  0.00244162 -0.00671473 -0.0379213 ], b: -0.0013075808375690545
cost at iteration 700: 689.7214026029069, w: [ 0.20343608  0.00276967 -0.00785347 -0.04497072],

#### feature and parameter values

* when the range is large $300 - 2000 $ then $w$ will be small
* when the range is small $1 - 5 $ then $w$ will be large

#### feature scaling

* make sure features are on a similar scale
* divide the input values by the range $(max - min)$ or by $max$



#### Mean normalization

$$ x_i = \dfrac{x_i - \mu_i}{s_i} $$

$$ \mu_i = \dfrac{1}{m} \sum_{j=1}^{m} x_i^{(j)} $$
$$ s_i = max(x_i) - min(x_i) $$

where:
* $x_i$ is the $i^{th}$ feature
* $\mu_i$ is the average of the $i^{th}$ feature
* $s_i$ is the range of the $i^{th}$ feature

#### z-score normalization:

$$ x_i = \dfrac{x_i - \mu_i}{\sigma_i} $$
$$ \sigma_i = \sqrt{\dfrac{1}{m} \sum_{j=1}^{m} (x_i^{(j)} - \mu_i)^2} $$

where:
* $x_i$ is the $i^{th}$ feature
* $\mu_i$ is the average of the $i^{th}$ feature
* $\sigma_i$ is the standard deviation of the $i^{th}$ feature


